In [38]:
import pandas as pd
import openpyxl, pprint, os, sys
from datetime import datetime, timedelta
from openpyxl.styles import Alignment
from dateutil.relativedelta import relativedelta

In [39]:
# csvファイルを読み込む
products = pd.read_csv("../input/products.csv", encoding="shift_jis")
orders = pd.read_csv("../input/orders.csv", encoding="shift_jis")

In [40]:
# テンプレートを読み込む
quotation_wb = openpyxl.load_workbook("../template/invoice_template.xlsx")
# 対象のワークシートを読み込む
quotation_ws = quotation_wb["Invoice"]

In [41]:
# 対象の注文IDを対話形式で入力
target_id = input("注文IDを入力して下さい")

注文IDを入力して下さいO0003


In [42]:
# 情報を元に見積書を作成
# 対象の取引データを抽出
target_orders = orders[orders["order_id"] == target_id]
target_orders

order_id customer_name order_date product_id  quantity  \
6    O0003           鈴木健  2025/5/11       P006         3   
7    O0003           鈴木健  2025/5/11       P002         5   

  invoice_registration_number  
6              T1234567890003  
7              T1234567890003

In [43]:
# データを結合
merged = target_orders.merge(products, on="product_id", how="left")
merged

order_id customer_name order_date product_id  quantity  \
0    O0003           鈴木健  2025/5/11       P006         3   
1    O0003           鈴木健  2025/5/11       P002         5   

  invoice_registration_number product_name  unit_price unit          note  \
0              T1234567890003        無線マウス        2500    個  2.4GHz ワイヤレス   
1              T1234567890003     レーザープリンタ       40000    台     両面印刷対応 A4   

   tax_rate category         supplier  
0        10     周辺機器   Peripheral Co.  
1        10     周辺機器  OfficeGear Inc.

In [44]:
# 金額列の小計
merged["subtotal"]  = merged["unit_price"] * merged["quantity"]

In [45]:
# 消費税
merged["VAT"] = merged["subtotal"] * merged["tax_rate"] / 100
merged["VAT"] = merged["VAT"].round(0).astype(int)

In [46]:
# 合計
merged["total_amount"] = merged["subtotal"] + merged["VAT"]
merged["total_amount"] = merged["total_amount"].astype(int)

In [47]:
merged

order_id customer_name order_date product_id  quantity  \
0    O0003           鈴木健  2025/5/11       P006         3   
1    O0003           鈴木健  2025/5/11       P002         5   

  invoice_registration_number product_name  unit_price unit          note  \
0              T1234567890003        無線マウス        2500    個  2.4GHz ワイヤレス   
1              T1234567890003     レーザープリンタ       40000    台     両面印刷対応 A4   

   tax_rate category         supplier  subtotal    VAT  total_amount  
0        10     周辺機器   Peripheral Co.      7500    750          8250  
1        10     周辺機器  OfficeGear Inc.    200000  20000        220000

In [48]:
# 請求書への入力作業
# 明細の開始行
start_row = 15

# 右詰めスタイルを作成
right_align = Alignment(horizontal="right")

# 日付設定
today = datetime.today()
next_month_same_day = today + relativedelta(months=1)

# 明細行描写
for i, (_, row) in enumerate(merged.iterrows()):
    # No(連番)
    quotation_ws.cell(row=start_row + i, column=1, value=i + 1) 
    # 商品名
    quotation_ws.cell(row=start_row + i, column=2, value=row["product_name"]) 
     # 数量
    quotation_ws.cell(row=start_row + i, column=3, value=row["quantity"])
    # 単価(円)
    cell = quotation_ws.cell(row=start_row + i, column=4, value=row["unit_price"]) 
    cell.number_format = '"¥"#,##0'
    # 税率
    quotation_ws.cell(row=start_row + i, column =5, value=str(row["tax_rate"])+"%")
    # 金額(円)
    cell = quotation_ws.cell(row=start_row + i, column=6, value=row["subtotal"]) 
    cell.number_format = '"¥"#,##0'
    
# 小計
cell = quotation_ws.cell(row=31, column=6, value=merged["subtotal"].sum()) 
cell.number_format = '"¥"#,##0'

# 消費税
cell = quotation_ws.cell(row=32, column=6, value=merged["VAT"].sum()) 
cell.number_format = '"¥"#,##0'

# 合計金額
cell = quotation_ws.cell(row=33, column=6, value=merged["total_amount"].sum()) 
cell.number_format = '"¥"#,##0'

# 請求金額
amount = merged["total_amount"].sum()
formatted = f"¥{amount:,.0f}(税込)"

cell = quotation_ws.cell(row=12, column=3, value=formatted)
cell.alignment = right_align

# 税率区分
# 10%
cell = quotation_ws.cell(row=36, column=4, 
                         value=merged[merged["tax_rate"] == 10]["VAT"].sum())
cell.number_format = '"¥"#,##0'

cell = quotation_ws.cell(row=36, column=5, 
                         value=merged[merged["tax_rate"] == 10]["subtotal"].sum())
cell.number_format = '"¥"#,##0'
cell.alignment = right_align

# 8%
cell = quotation_ws.cell(row=37, column=4, 
                         value=merged[merged["tax_rate"] == 8]["VAT"].sum())
cell.number_format = '"¥"#,##0'

cell = quotation_ws.cell(row=37, column=5, 
                         value=merged[merged["tax_rate"] == 8]["subtotal"].sum())
cell.number_format = '"¥"#,##0'
cell.alignment = right_align

# 宛名
quotation_ws["A4"] = merged.iloc[0]["customer_name"] + "  様"

# 発行日
cell = quotation_ws.cell(row=5, column=6, value=today.strftime("%Y年%m月%d日")) 
cell.alignment = right_align

# 登録番号
cell = quotation_ws.cell(row=6, column=6, 
                         value=merged["invoice_registration_number"].iloc[0]) 
cell.alignment = right_align

# 請求番号
cell = quotation_ws.cell(row=7, column=6, 
                         value=merged["order_id"].iloc[0]) 
cell.alignment = right_align

# お支払期日
cell = quotation_ws.cell(row=42, column=3, 
                         value=next_month_same_day.strftime("%Y年%m月末"))
cell.alignment = right_align
    
quotation_wb.save(f"../output/{target_id}_invoice.xlsx")